#### Prerequisite

In [1]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


#### Create a docker image

This step can run in SageMaker classic notebook environment or in your own local environment with docker installed.

In [2]:
# Image name below preferably starts with the prefix `sagemaker`
!docker build -t sagemaker_text_encoder -f Dockerfile .

Sending build context to Docker daemon  1.443MB
Step 1/7 : FROM python:3.7
 ---> 5cda39795abb
Step 2/7 : COPY requirements.txt ./
 ---> 1588774ea606
Step 3/7 : RUN python -m pip install --upgrade pip
 ---> Running in 071f33c43e81
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Removing intermediate container 071f33c43e81
 ---> 9785af30ece0
Step 4/7 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Running in 8a91fce1a47b
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.1/804.1 MB 163.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 185.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 264.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 218.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 188.1 MB/s e

#### Push docker image from local to ECR

In [3]:
%%sh

# Specify a name to your custom container
container_name=sagemaker_text_encoder  # should match name from previous cell
echo "Container Name: " ${container_name}

# Retreive AWS account ID
account=$(aws sts get-caller-identity --query Account --output text)

# Get the AWS region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: " ${account}
echo "Region: "${region}

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "ECR Repository: " ${repository}

image="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:latest"
echo "ECR Image URI: " ${image}

# If the ECR repository does not exist, create it.
aws ecr describe-repositories --repository-names ${container_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${container_name} > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Tag the local image with ECR image name
docker tag ${container_name} ${image}

# Finally, push the local docker image to ECR with the full ECR image name
docker push ${image}

Container Name:  sagemaker_text_encoder
Account:  119174016168
Region: us-east-1
ECR Repository:  119174016168.dkr.ecr.us-east-1.amazonaws.com
ECR Image URI:  119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker_text_encoder:latest
Login Succeeded
The push refers to repository [119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker_text_encoder]
7b4903b5f373: Preparing
d88ab4d0756e: Preparing
e755b640ef85: Preparing
64c5ff71ede4: Preparing
ffc335b17c21: Preparing
eef135e35b6e: Preparing
a0db21004f62: Preparing
0b53caaeb40b: Preparing
1cad4dc57058: Preparing
4ff8844d474a: Preparing
b77487480ddb: Preparing
cd247c0fb37b: Preparing
cfdd5c3bd77e: Preparing
870a241bfebd: Preparing
0b53caaeb40b: Waiting
1cad4dc57058: Waiting
4ff8844d474a: Waiting
b77487480ddb: Waiting
cd247c0fb37b: Waiting
cfdd5c3bd77e: Waiting
870a241bfebd: Waiting
eef135e35b6e: Waiting
a0db21004f62: Waiting
d88ab4d0756e: Pushed
ffc335b17c21: Pushed
7b4903b5f373: Pushed
a0db21004f62: Pushed
64c5ff71ede4: Pushed
eef135e35b6

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



##### Uncomment and run the command below if your local images need to be cleaned.

In [4]:
#!docker rmi $(docker images -q) -f

## Deploy feature encoder as a SageMaker real time endpoint

#### Imports 

In [5]:
from time import gmtime, strftime
import sagemaker
import datetime
import boto3
import time

#### Essentials

In [6]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
account = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name
sagemaker_client = boto3.client('sagemaker', region_name=region)
image_name = 'sagemaker_text_encoder'
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest'
current_timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

#### Pack all encoders into a tar file and push to S3

In [9]:
!tar -czf text-encoder.tar.gz -C ./data .

In [10]:
default_bucket = session.default_bucket()
default_bucket

'sagemaker-us-east-1-119174016168'

In [11]:
!aws s3 cp text-encoder.tar.gz s3://{default_bucket}/

upload: ./text-encoder.tar.gz to s3://sagemaker-us-east-1-119174016168/text-encoder.tar.gz


#### Create a SageMaker model object

In [14]:
model_name = f'text-encoder-{current_timestamp}'
model_artifacts_location = f's3://{default_bucket}/text-encoder.tar.gz'

In [15]:
response = sagemaker_client.create_model(ModelName=model_name, 
                                         Containers=[{
                                             'Image': image_uri, 
                                             'Mode': 'SingleModel', 
                                             'ModelDataUrl': model_artifacts_location}], 
                                         ExecutionRoleArn=role)
response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:119174016168:model/text-encoder-2022-12-16-19-35-54',
 'ResponseMetadata': {'RequestId': '950330b0-d7a0-4de6-b87a-36e0c6d6f728',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '950330b0-d7a0-4de6-b87a-36e0c6d6f728',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '94',
   'date': 'Fri, 16 Dec 2022 19:37:14 GMT'},
  'RetryAttempts': 0}}

#### Create a SageMaker endpoint configuration

In [16]:
endpoint_config_name = f'text-encoder-{current_timestamp}'

In [17]:
response = sagemaker_client.create_endpoint_config(EndpointConfigName=endpoint_config_name, 
                                                   ProductionVariants=[{
                                                       'VariantName': 'v1', 
                                                       'ModelName': model_name, 
                                                       'InstanceType': 'ml.c5.xlarge', 
                                                       'InitialInstanceCount': 2, 
                                                       'InitialVariantWeight': 1
                                                   }])
response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:119174016168:endpoint-config/text-encoder-2022-12-16-19-35-54',
 'ResponseMetadata': {'RequestId': '06df06d7-b64c-49ea-96c5-19437a505c3a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '06df06d7-b64c-49ea-96c5-19437a505c3a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '113',
   'date': 'Fri, 16 Dec 2022 19:37:20 GMT'},
  'RetryAttempts': 0}}

#### Create a SageMaker endpoint

In [18]:
endpoint_name = f'text-encoder-{current_timestamp}'

In [19]:
response = sagemaker_client.create_endpoint(EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)
response

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:119174016168:endpoint/text-encoder-2022-12-16-19-35-54',
 'ResponseMetadata': {'RequestId': 'af328c83-6ade-4504-9fe5-fc288fbcc9ea',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'af328c83-6ade-4504-9fe5-fc288fbcc9ea',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Fri, 16 Dec 2022 19:37:28 GMT'},
  'RetryAttempts': 0}}

#### Describe endpoint to track creation status

In [20]:
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = response['EndpointStatus']

while endpoint_status == 'Creating':
    time.sleep(15)
    response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = response['EndpointStatus'] 
    print(endpoint_status)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


#### Invoke endpoint to test deployed feature encoder

In [21]:
sagemaker_runtime = boto3.client('sagemaker-runtime', 
                                 region_name=region)

In [22]:
raw_payload = b"I purchased this headphones on Black Friday sale. Poor quality. Not worth the money."

In [23]:
%%time

response = sagemaker_runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                             Body=raw_payload, 
                                             ContentType='text/csv')
response

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/text-encoder-2022-12-16-19-35-54 in account 119174016168 for more information.

##### Extract feature vector from the response

In [24]:
feature_vector = response['Body'].read().decode('utf-8').strip()
feature_vector

KeyError: 'Body'